This is a script for pre-processing patient-pharmacy visit data.

In [16]:
import os
import configparser
import warnings
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None 

Define the folder paths.

In [2]:
BASE_DIR = os.getcwd()
CONFIG = configparser.ConfigParser()
CONFIG.read(os.path.join(BASE_DIR, 'script_config.ini'))

BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

DATA_RAW = os.path.join(BASE_PATH, 'raw')
DATA_RESULTS = os.path.join(BASE_PATH, '..', 'results')

Read the raw pharmacy data.

In [25]:
data_path = os.path.join(DATA_RAW, 'avondale_pharmacies.csv')
df = pd.read_csv(data_path)
df = df[['Pts adrees', 'PatAge', 'PatRace', 'PatInsuranceType', 
         'PatSex', 'PatPrefPharmZIP', 'PatPrefPharmacyName',
         'AltevPrefferedPharmacyFlag Tru']]
df['counts'] = 1

We preprocess the age distribution by grouping them into bins. 

In [34]:
bins = [18, 35, 45, 60, 75, float('inf')]
labels = ['18 - 35 years', '36 - 45 years', 
          '46 - 60 years', '61 - 75 years', 'Over 75 years']
df['age_group'] = pd.cut(df['PatAge'], bins = bins, labels = 
            labels, right = True, include_lowest = True)

In [49]:
data_path = os.path.join(DATA_RESULTS, 'pharmacies_geocoded.csv')
dff = pd.read_csv(data_path)
bins = [0, 10, 40, float('inf')]
labels = ['1 to 10', '11 to 40', 'Over 40']
dff['visit_count'] = pd.cut(dff['counts'], bins = bins, labels = 
            labels, right = True, include_lowest = True)
filename = 'pharmacies_geocoded.csv'
path_out = os.path.join(DATA_RESULTS, filename)
dff.to_csv(path_out, index = False)

## Group by names

In [30]:
df1 = df.groupby(['PatPrefPharmZIP', 'PatPrefPharmacyName',])['counts'].sum().reset_index()
df1['state'] = 'Ohio'

#### Address Geocoding

First we initialize the geocode and include the rate limiter to avoid getting blocked.

In [27]:
geolocator = Nominatim(user_agent = 'pharmacy_geocoder', timeout = 10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds = 2,        
    max_retries = 3, error_wait_seconds=5.0)
#df1['address'] = df1['Pts adrees'].astype(str)

In [28]:
tqdm.pandas()
df1['location'] = df1['PatPrefPharmacyName'].progress_apply(geocode)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 88/88 [03:32<00:00,  2.42s/it]


We then extract the latitude and longitudes.

In [29]:
df1['latitude'] = df1['location'].apply(lambda loc: loc.latitude if loc else None)
df1['longitude'] = df1['location'].apply(lambda loc: loc.longitude if loc else None)
filename = 'pharmacy_names.csv'
path_out = os.path.join(DATA_RESULTS, filename)
df1.to_csv(path_out, index = False)

In [31]:
path_out = os.path.join(DATA_RESULTS, filename)
df1.to_csv(path_out, index = False)

## Group by Race

In [35]:
df2 = df.groupby(['PatRace', 'age_group'])['counts'].sum().reset_index()

filename = 'pharmacy_race.csv'
path_out = os.path.join(DATA_RESULTS, filename)
df2.to_csv(path_out, index = False)

## Group by Sex

In [36]:
df3 = df.groupby(['PatSex', 'age_group'])['counts'].sum().reset_index()

filename = 'pharmacy_sex.csv'
path_out = os.path.join(DATA_RESULTS, filename)
df3.to_csv(path_out, index = False)

## Group by Age

In [38]:
df4 = df.groupby(['PatInsuranceType', 'age_group'])['counts'].sum().reset_index()

filename = 'pharmacy_age.csv'
path_out = os.path.join(DATA_RESULTS, filename)
df4.to_csv(path_out, index = False)